# Prefix
In this notebook, we will be looking at the happy-or-sad dataset and create a model that would predict if the person is happy or sad.

In [0]:
import tensorflow as tf
import os, zipfile

# Step 1: Loading the dataset

In [2]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2019-06-17 19:09:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 2404:6800:4003:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2019-06-17 19:09:19 (191 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



# Step 2: Building the Model
We are going to build the model with multiply convolutional layers and maxpooling layers.

### Custom Callbacks function
A custom callbacks function stop the training at 99.9% accuracy.

In [0]:
# creating callbacks to stop training at 99.9%
class myCallbacks(tf.keras.callbacks.Callback):
  def __init__(self, desired_accuracy=.999):
    self.d_acc = desired_accuracy
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('acc') > self.d_acc:
      print('\nAccuracy above {:.2f}%, Training STOPPED!'.format(self.d_acc))
      self.model.stop_training = True

### Building the Convolutional Model

In [0]:
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model

In [0]:
# helper function for creating multiply conv2d and maxpooling layers
def multi_conv(x, size):
  assert(isinstance(size, list)), 'size arg must be a list containing integers'
  for s in size:
    l = Conv2D(s, (3,3), activation='relu')(x)
    l = MaxPooling2D(2,2)(l)
  return l

In [6]:
# building the model with multi convolutional layers with maxpooling
in_x = Input(shape=(300, 300, 3))
X = multi_conv(in_x, [16, 32, 64, 32, 64, 32])
X = Flatten()(X)
X = Dense(512, activation='relu')(X)
X = Dense(1, activation='sigmoid')(X)

model = Model(in_x, X)

model.summary()

W0617 19:09:47.010967 139731058648960 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 710432)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               363741696 
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 363,743,105
Trainable params: 363,743,105
Non-trainable params: 0
_______________________________________________

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

W0617 19:09:51.941649 139731058648960 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Step 3: Preprocessing the Data
We are going to use the ImageDataGenerator function from keras.preprocessing to how us easily preprocess the image data.

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory('/tmp/h-or-s',
                                                    target_size=(300, 300),
                                                    batch_size=10,
                                                    class_mode='binary')

Found 80 images belonging to 2 classes.


# Step 4: Training the Model

In [10]:
# setting callbacks
callbacks = myCallbacks()
# training the model
model.fit_generator(train_generator, 
                    epochs=15,
                    steps_per_epoch=2,
                    verbose=1,
                    callbacks=[callbacks])

Epoch 1/15
2/2 [==============================] - 12s 6s/step - loss: 100.1854 - acc: 0.7500
Epoch 2/15
2/2 [==============================] - 0s 100ms/step - loss: 89.6502 - acc: 0.4000
Epoch 3/15
2/2 [==============================] - 0s 99ms/step - loss: 14.9197 - acc: 0.3000
Epoch 4/15
2/2 [==============================] - 0s 99ms/step - loss: 0.4519 - acc: 0.8500
Epoch 5/15
1/2 [==============>...............] - ETA: 0s - loss: 0.0029 - acc: 1.0000
Accuracy above 1.00%, Training STOPPED!
2/2 [==============================] - 0s 150ms/step - loss: 0.0014 - acc: 1.0000
